# Рекомендация тарифов

**Описание проекта.**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

**Цель исследования** - Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

**Описание данных.**
 - `сalls` — количество звонков
 - `minutes` — суммарная длительность звонков в минутах
 - `messages` — количество sms-сообщений
 - `mb_used` — израсходованный интернет-трафик в Мб
 - `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from sklearn.datasets import load_iris
from sklearn import tree
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv('/DS/Yandex_Practikum/Projects/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


имеем 3214 строк без нулевых значений

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбейте данные на выборки

разделяю ДФ на три выборки тренировочная, валидная, тестовая в соотношении 2:1:1. Целевой признак столбец 'is_ultra'

In [4]:
features = df.drop('is_ultra',axis = 1)
target = df['is_ultra'] 

In [5]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.5, random_state=12345, stratify = target)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

In [6]:
features_train.shape

(1607, 4)

## Исследуйте модели

In [7]:
best_model = None
best_result = 0

случайный лес

In [8]:
%%time
for est in range(1, 50):
    for depth in range (1,10,1):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) # обучите модель с заданным количеством деревьев
        model.fit(features_train,target_train) # обучите модель на тренировочной выборке
        result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
        if result > best_result:
            best_model = model# сохраните наилучшую модель
            best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных

CPU times: user 21.4 s, sys: 195 ms, total: 21.6 s
Wall time: 21.6 s


In [9]:
best_result

0.8156911581569116

логистическая регрессия

In [10]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train,target_train) 
result = model.score(features_valid, target_valid)
print(result)
if result > best_result:
    best_model = model# сохраните наилучшую модель
    best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных

0.7397260273972602


In [11]:
best_result

0.8156911581569116

дерево решений

In [12]:
for depth in range(1,6,1):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train,target_train) # обучите модель
    predictions_valid = model.predict(features_valid)
    result = model.score(features_valid,target_valid)
    if result > best_result:
        best_model = model# сохраните наилучшую модель
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных

In [13]:
best_result

0.8156911581569116

In [14]:
#fig = plt.figure(figsize=(35,20))
#_ = tree.plot_tree(model, 
#               feature_names=features.columns.values, 
#               class_names=['Smart','Ultra'],
#               filled=True,
 #              fontsize = 15)

## Проверьте модель на тестовой выборке

In [15]:
best_model

RandomForestClassifier(max_depth=9, n_estimators=19, random_state=12345)

In [16]:
best_result

0.8156911581569116

In [17]:
result = model.score(features_test, target_test)
result

0.7960199004975125

## (бонус) Проверьте модели на адекватность

In [18]:
from sklearn.dummy import DummyClassifier

In [20]:
dummy_clf = DummyClassifier(strategy="most_frequent")
predictions_test = [0]*target_test.shape[0]
dummy_clf.fit(features_train, target_train)
dummy_clf.predict(features_valid)
dummy_clf.score(features_test,target_test)

0.6890547263681592

## Вывод:

- Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
- В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 
___
- По представленным данным было обучено три модели _RandomForestClassifier, LogisticRegression, DecisionTreeClassifier_ и выбрана лучшая из них - **RandomForestClassifier** с количеством деревьев 19 и глубиной 9. Точность данной модели на валидационной выборке - **0.815**
___
- Точность на тестовой выборке - **0.796**. Что удовлетворяет условию точности не меньше **0.75**
